# Multiple Myloma Patients

In [1]:
import qgrid
import pandas as pd
from drg_connect import Snowflake
 
snow = Snowflake(role = 'RWD_ANALYTICS_RW', database='PROJECT_ANALYTICS',schema ='TRAINING')
engine = snow.engine
#%reload_ext sql_magic  
%load_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# Pulling MM Patients from Combined EHR and Holmusk Table

## Joining MM EHR tables and Holmusk EHR tables 

In [3]:
%%read_sql 
Create or replace Table MM_EHR_COM AS
SELECT DISTINCT 
   -- CLAIM_NUMBER,
   'Albatross' as Data_source,
    ENCRYPTED_KEY_1,
    ENCRYPTED_KEY_2,   
     year_of_service
  
FROM  Myeloma_Albatross_EHR
union
SELECT DISTINCT 
   -- CLAIM_NUMBER,
   'Pelican' as Data_source,
    ENCRYPTED_KEY_1,
    ENCRYPTED_KEY_2,   
     year_of_service
  
FROM  Myeloma_Pelican_EHR 
union
SELECT DISTINCT 
    'Holmusk' as Data_source,
   -- CLAIM_NUMBER,
    ENCRYPTED_KEY_1,
    ENCRYPTED_KEY_2,   
    Encounter_date as year_of_service
    
From  holmusk_EHR_encounter_20180712_final 
Where ENCRYPTED_KEY_1 IS NOT NULL
    AND     TRIM(ENCRYPTED_KEY_1) <> ''
    AND     UPPER(ENCRYPTED_KEY_1) NOT LIKE 'XXX -%'
    AND     UPPER(ENCRYPTED_KEY_1) <> 'NULL'
    AND     ENCRYPTED_KEY_2 IS NOT NULL
    AND     TRIM(ENCRYPTED_KEY_2) <> ''
    AND     UPPER(ENCRYPTED_KEY_2) NOT LIKE 'XXX -%'
    AND     UPPER(ENCRYPTED_KEY_2) <> 'NULL'
    AND     YEAR(encounter_date) BETWEEN 2014 AND 2018 
  



Query started at 03:17:25 PM India Standard Time; Query executed in 0.83 m

,status
0,Table MM_EHR_COM successfully created.


In [8]:
%%read_sql
select
distinct
data_source,
count(distinct encrypted_key_1,encrypted_key_2),
count( *)

from MM_EHR_COM 
group by 1

Query started at 03:31:39 PM India Standard Time; Query executed in 0.14 m

,data_source,"COUNT(DISTINCT ENCRYPTED_KEY_1,ENCRYPTED_KEY_2)",COUNT( *)
0,Pelican,12565,15299
1,Albatross,24897,214128
2,Holmusk,1593366,73083279


### Pulling All MM patients records from MM_EHR_COM

In [19]:
%%read_sql
Create or replace table MM_Patients_from_EHR as
Select distinct a.encrypted_key_1,
       a.encrypted_key_2,
       a.year_of_service,
       b.data_source
From  Myeloma_patients a
inner join MM_EHR_COM b
on a.encrypted_key_1||a.encrypted_key_2=b.encrypted_key_1||b.encrypted_key_2

where   a.ENCRYPTED_KEY_1 IS NOT NULL
    AND     TRIM(a.ENCRYPTED_KEY_1) <> ''
    AND     UPPER(a.ENCRYPTED_KEY_1) NOT LIKE 'XXX -%'
    AND     UPPER(a.ENCRYPTED_KEY_1) <> 'NULL'
    AND     a.ENCRYPTED_KEY_2 IS NOT NULL
    AND     TRIM(a.ENCRYPTED_KEY_2) <> ''
    AND     UPPER(a.ENCRYPTED_KEY_2) NOT LIKE 'XXX -%'
    AND     UPPER(a.ENCRYPTED_KEY_2) <> 'NULL'
    AND     YEAR(a.year_of_service) BETWEEN 2014 AND 2018  
    
      


Query started at 05:00:45 PM India Standard Time; Query executed in 0.29 m

,status
0,Table MM_PATIENTS_FROM_EHR successfully created.


In [20]:
%%read_sql
select
distinct
data_source,
count(distinct encrypted_key_1,encrypted_key_2),
count( *)

from MM_PATIENTS_FROM_EHR  
group by 1

Query started at 05:01:30 PM India Standard Time; Query executed in 0.07 m

,data_source,"COUNT(DISTINCT ENCRYPTED_KEY_1,ENCRYPTED_KEY_2)",COUNT( *)
0,Pelican,12565,320166
1,Albatross,24897,799887
